In [1]:
import itertools
import os
import sys
from itertools import combinations_with_replacement
from math import e
import gc

import numpy as np
import pandas as pd
from numba import njit
from plotly.subplots import make_subplots
from vectorbt.indicators import nb
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import vectorbt as vbt
from lib.utils import file_to_data_frame, LR, ExtendedPortfolio, plot_series_vs_scatters, dropnaninf, get_best_index

In [2]:
file = "/Users/pilo/development/itba/pf/Binance_Minute_OHLC_CSVs/shorts/Binance_BTCUSDT_minute_3000.csv"
_, ohlcv = file_to_data_frame(file)
ohlcv. head()

,Open,High,Low,Close,Volume BTC,Volume,Tradecount
date,,,,,,,
2021-03-16 22:08:00,56669.99,56700.00,56666.32,56676.18,44.050583,2.497137e+06,1532
2021-03-16 22:09:00,56676.18,56713.70,56624.73,56699.99,47.402111,2.686370e+06,1380
2021-03-16 22:10:00,56699.99,56740.57,56680.69,56732.66,60.198143,3.413590e+06,1476
2021-03-16 22:11:00,56732.66,56739.83,56658.45,56700.00,47.334721,2.683621e+06,1329
2021-03-16 22:12:00,56700.01,56751.50,56700.00,56748.48,27.110728,1.538395e+06,1107


In [3]:
close = ohlcv["Close"]
volume = ohlcv["Volume"]
lr_ind = LR.run(close)
print(lr_ind.lr.shape)

(2999,)


In [4]:
lag = list(range(10,12))
ma_ind = vbt.MA.run(lr_ind.lr, lag, short_name="lr_ma")
mstd_ind = vbt.MSTD.run(lr_ind.lr, lag)

In [5]:
@njit
def combination_nb(ma, std, thld):
    return ma + thld * std

LR_MULTIPLIER = vbt.IndicatorFactory(
    input_names=['ma', 'std'],
    param_names=['thld'],
    output_names=['mu']
).from_apply_func(combination_nb, use_ray=True)
thld_granularity = 10
thld = np.linspace(0,3.5, thld_granularity, endpoint=True)
ups = LR_MULTIPLIER.run(ma_ind.ma, mstd_ind.mstd, thld=thld, short_name="ups_mu")
pitfalls = LR_MULTIPLIER.run(ma_ind.ma, mstd_ind.mstd, thld=-thld, short_name="pitfalls_mu")
pitfalls.mu.head()

pitfalls_mu_thld    -0.000000     -0.388889     -0.777778     -1.166667      \
mstd_window                10  11        10  11        10  11        10  11   
date                                                                          
2021-03-16 22:08:00       NaN NaN       NaN NaN       NaN NaN       NaN NaN   
2021-03-16 22:09:00       NaN NaN       NaN NaN       NaN NaN       NaN NaN   
2021-03-16 22:10:00       NaN NaN       NaN NaN       NaN NaN       NaN NaN   
2021-03-16 22:11:00       NaN NaN       NaN NaN       NaN NaN       NaN NaN   
2021-03-16 22:12:00       NaN NaN       NaN NaN       NaN NaN       NaN NaN   

pitfalls_mu_thld    -1.555556     -1.944444     -2.333333     -2.722222      \
mstd_window                10  11        10  11        10  11        10  11   
date                                                                          
2021-03-16 22:08:00       NaN NaN       NaN NaN       NaN NaN       NaN NaN   
2021-03-16 22:09:00       NaN NaN       NaN NaN       NaN NaN       NaN NaN   
2021-03-16 22:10:00       NaN NaN       NaN NaN       NaN NaN       NaN NaN   
2021-03-16 22:11:00       NaN NaN       NaN NaN       NaN NaN       NaN NaN   
2021-03-16 22:12:00       NaN NaN       NaN NaN       NaN NaN       NaN NaN   

pitfalls_mu_thld    -3.111111     -3.500000      
mstd_window                10  11        10  11  
date                                             
2021-03-16 22:08:00       NaN NaN       NaN NaN  
2021-03-16 22:09:00       NaN NaN       NaN NaN  
2021-03-16 22:10:00       NaN NaN       NaN NaN  
2021-03-16 22:11:00       NaN NaN       NaN NaN  
2021-03-16 22:12:00       NaN NaN       NaN NaN

In [6]:
lr_entries = lr_ind.lr_below(pitfalls.mu)
lr_exits = lr_ind.lr_above(ups.mu)
lr_entries.shape

(2999, 20)

In [7]:
@njit
def multiplier_nb(values, m):
    return m * values

MULTIPLIER = vbt.IndicatorFactory(
    input_names=['values'],
    param_names=['m'],
    output_names=['mu']
).from_apply_func(multiplier_nb)

In [8]:
# Volume part:
vol_ma = vbt.MA.run(volume, lag)
vol_multiplier = MULTIPLIER.run(vol_ma.ma, thld, short_name="vol_mu")
vol_entries = vol_multiplier.mu_below(volume)
print(vol_entries.sum(axis=0))
print("shape", vol_entries.shape)

vol_mu_m  ma_window
0.000000  10           2990
          11           2989
0.388889  10           2913
          11           2902
0.777778  10           1868
          11           1857
1.166667  10            797
          11            790
1.555556  10            365
          11            366
1.944444  10            176
          11            187
2.333333  10             94
          11            101
2.722222  10             53
          11             54
3.111111  10             27
          11             29
3.500000  10             17
          11             19
dtype: int64
shape (2999, 20)


In [9]:
if len(lr_entries.shape) > 1:
    lr_entries.columns = lr_entries.columns.rename("lag", level=-1)
    lr_entries.columns = lr_entries.columns.reorder_levels([-1, 0])
    vol_entries.columns = vol_entries.columns.rename("lag", level=-1)
    vol_entries.columns = vol_entries.columns.reorder_levels([-1, 0])

In [17]:
vol_col = vol_entries.columns.get_level_values("vol_mu_m").unique()
lr_col = lr_entries.columns.get_level_values("pitfalls_mu_thld").unique()
new_lr_entries= lr_entries.vbt.tile(n=len(vol_col), keys=vol_col)
new_lr_entries.head()

vol_mu_m                  0.0                                          \
lag                        10        11        10        11        10   
pitfalls_mu_thld    -0.000000 -0.000000 -0.388889 -0.388889 -0.777778   
date                                                                    
2021-03-16 22:08:00     False     False     False     False     False   
2021-03-16 22:09:00     False     False     False     False     False   
2021-03-16 22:10:00     False     False     False     False     False   
2021-03-16 22:11:00     False     False     False     False     False   
2021-03-16 22:12:00     False     False     False     False     False   

vol_mu_m                                                               ...  \
lag                        11        10        11        10        11  ...   
pitfalls_mu_thld    -0.777778 -1.166667 -1.166667 -1.555556 -1.555556  ...   
date                                                                   ...   
2021-03-16 22:08:00     False     False     False     False     False  ...   
2021-03-16 22:09:00     False     False     False     False     False  ...   
2021-03-16 22:10:00     False     False     False     False     False  ...   
2021-03-16 22:11:00     False     False     False     False     False  ...   
2021-03-16 22:12:00     False     False     False     False     False  ...   

vol_mu_m                  3.5                                          \
lag                        10        11        10        11        10   
pitfalls_mu_thld    -1.944444 -1.944444 -2.333333 -2.333333 -2.722222   
date                                                                    
2021-03-16 22:08:00     False     False     False     False     False   
2021-03-16 22:09:00     False     False     False     False     False   
2021-03-16 22:10:00     False     False     False     False     False   
2021-03-16 22:11:00     False     False     False     False     False   
2021-03-16 22:12:00     False     False     False     False     False   

vol_mu_m                                                               
lag                        11        10        11        10        11  
pitfalls_mu_thld    -2.722222 -3.111111 -3.111111 -3.500000 -3.500000  
date                                                                   
2021-03-16 22:08:00     False     False     False     False     False  
2021-03-16 22:09:00     False     False     False     False     False  
2021-03-16 22:10:00     False     False     False     False     False  
2021-03-16 22:11:00     False     False     False     False     False  
2021-03-16 22:12:00     False     False     False     False     False  

[5 rows x 200 columns]

In [18]:
new_vol_entries = vol_entries.vbt.tile(n=len(lr_col), keys=lr_col)
new_vol_entries[:20]

pitfalls_mu_thld        -0.0                                               \
lag                       10       11       10       11       10       11   
vol_mu_m            0.000000 0.000000 0.388889 0.388889 0.777778 0.777778   
date                                                                        
2021-03-16 22:08:00    False    False    False    False    False    False   
2021-03-16 22:09:00    False    False    False    False    False    False   
2021-03-16 22:10:00    False    False    False    False    False    False   
2021-03-16 22:11:00    False    False    False    False    False    False   
2021-03-16 22:12:00    False    False    False    False    False    False   
2021-03-16 22:13:00    False    False    False    False    False    False   
2021-03-16 22:14:00    False    False    False    False    False    False   
2021-03-16 22:15:00    False    False    False    False    False    False   
2021-03-16 22:16:00    False    False    False    False    False    False   
2021-03-16 22:17:00     True    False     True    False    False    False   
2021-03-16 22:18:00     True     True     True     True     True     True   
2021-03-16 22:19:00     True     True     True     True     True     True   
2021-03-16 22:20:00     True     True     True     True     True     True   
2021-03-16 22:21:00     True     True     True     True     True     True   
2021-03-16 22:22:00     True     True     True     True    False    False   
2021-03-16 22:23:00     True     True     True     True     True     True   
2021-03-16 22:24:00     True     True     True     True     True     True   
2021-03-16 22:25:00     True     True     True     True     True     True   
2021-03-16 22:26:00     True     True     True     True    False    False   
2021-03-16 22:27:00     True     True     True     True    False    False   

pitfalls_mu_thld                                         ...     -3.5  \
lag                       10       11       10       11  ...       10   
vol_mu_m            1.166667 1.166667 1.555556 1.555556  ... 1.944444   
date                                                     ...            
2021-03-16 22:08:00    False    False    False    False  ...    False   
2021-03-16 22:09:00    False    False    False    False  ...    False   
2021-03-16 22:10:00    False    False    False    False  ...    False   
2021-03-16 22:11:00    False    False    False    False  ...    False   
2021-03-16 22:12:00    False    False    False    False  ...    False   
2021-03-16 22:13:00    False    False    False    False  ...    False   
2021-03-16 22:14:00    False    False    False    False  ...    False   
2021-03-16 22:15:00    False    False    False    False  ...    False   
2021-03-16 22:16:00    False    False    False    False  ...    False   
2021-03-16 22:17:00    False    False    False    False  ...    False   
2021-03-16 22:18:00    False    False    False    False  ...    False   
2021-03-16 22:19:00    False    False    False    False  ...    False   
2021-03-16 22:20:00    False    False    False    False  ...    False   
2021-03-16 22:21:00    False    False    False    False  ...    False   
2021-03-16 22:22:00    False    False    False    False  ...    False   
2021-03-16 22:23:00     True     True    False    False  ...    False   
2021-03-16 22:24:00     True     True     True     True  ...    False   
2021-03-16 22:25:00     True     True     True     True  ...    False   
2021-03-16 22:26:00    False    False    False    False  ...    False   
2021-03-16 22:27:00    False    False    False    False  ...    False   

pitfalls_mu_thld                                                           \
lag                       11       10       11       10       11       10   
vol_mu_m            1.944444 2.333333 2.333333 2.722222 2.722222 3.111111   
date                                                                        
2021-03-16 22:08:00    False    False    False    False    False    False   
2021-03-16 22

In [19]:
final_entries = new_lr_entries.vbt & new_vol_entries
final_entries.columns[:50]

MultiIndex([(               0.0,                -0.0, ...),
            (               0.0,                -0.0, ...),
            (               0.0, -0.3888888888888889, ...),
            (               0.0, -0.3888888888888889, ...),
            (               0.0, -0.7777777777777778, ...),
            (               0.0, -0.7777777777777778, ...),
            (               0.0, -1.1666666666666667, ...),
            (               0.0, -1.1666666666666667, ...),
            (               0.0, -1.5555555555555556, ...),
            (               0.0, -1.5555555555555556, ...),
            (               0.0, -1.9444444444444444, ...),
            (               0.0, -1.9444444444444444, ...),
            (               0.0, -2.3333333333333335, ...),
            (               0.0, -2.3333333333333335, ...),
            (               0.0, -2.7222222222222223, ...),
            (               0.0, -2.7222222222222223, ...),
            (               0.0,  -3.111

In [ ]:
new_lr_exits = lr_exits.vbt.repeat(n=len(lr_col))

In [ ]:
int_final_entries = pd.DataFrame(np.where(final_entries == True, 1, np.nan))
_all_signals =  pd.DataFrame(np.where(new_lr_exits == True, -1, int_final_entries), index=final_entries.index, columns=final_entries.columns)
del int_final_entries
_all_signals.shape

In [ ]:
@njit
def k_mean(col, arr, *args):
    indexes = np.where(np.isfinite(arr))[0]
    lr = args[0]
    n = 1
    adder = 0
    counter = 0
    while n < len(indexes):
        i = indexes[n]
        prev = indexes[n-1]
        if arr[prev] == 1 and arr[i] == -1:
            adder += np.mean(lr[prev+1:i +1])
            counter += 1
        n +=1
    return adder / counter if counter != 0 else 0
entry_exit_lr_avg = _all_signals.vbt.reduce(k_mean, lr_ind.lr.to_numpy())
entry_exit_lr_avg.head()

In [ ]:
entry_exit_lr_avg.vbt.volume().show()

In [ ]:
@njit
def double_multiplier_nb(values, x, y):
    return values*x, values*y

DOUBLE_MULTIPLIER = vbt.IndicatorFactory(
    input_names=['values'],
    param_names=['x', 'y'],
    output_names=['x_mu','y_mu']
).from_apply_func(double_multiplier_nb)
x = np.linspace(0,2,5, endpoint=True)
y = -np.linspace(0,2,5, endpoint=True)
tp_sl = DOUBLE_MULTIPLIER.run(mstd_ind.mstd, x, y, param_product=True, short_name="tp_sl")

In [ ]:
tp_exits = lr_ind.lr_above(tp_sl.x_mu)
sl_exits = lr_ind.lr_below(tp_sl.y_mu)
final_exits = tp_exits.vbt | sl_exits.vbt
final_exits.columns = final_exits.columns.rename("lag", level=-1)
final_exits.head()

In [ ]:
portfolio_kwargs = dict(
    direction='longonly',
    freq='m',
)
port = ExtendedPortfolio.from_signals(close, entries=final_entries, exits=final_exits, **portfolio_kwargs, max_logs=0)

In [ ]:
elr = port.expected_log_returns()
top_elr = elr.nlargest(25, keep="all")

In [ ]:
sharpe = dropnaninf(port.sharpe_ratio())
top_sharpe = sharpe.nlargest(25, keep="all")

In [ ]:
top_elr.index.intersection(top_sharpe.index)

In [ ]:
port.stats(column=top_elr.index[0])

In [ ]:
port.stats(column=top_sharpe.index[0])

In [ ]:
final_entries_plot = (final_entries.where(final_entries == True, np.nan)).vbt.scatterplot()
lr_plot = plot_series_vs_scatters([lr_ind.lr, pitfalls.mu], [lr_entries, lr_exits])
vol_plot = plot_series_vs_scatters([vol_multiplier.mu, volume], [vol_entries])

In [ ]:
def add_all_subplots(fig, row, col, list):
    for a in list:
        fig.add_trace(a, row=row, col=col)

In [ ]:
fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                    vertical_spacing=0.02)
add_all_subplots(fig, 1, 1, lr_plot.data)
add_all_subplots(fig, 2, 1, vol_plot.data)
add_all_subplots(fig, 3, 1, final_entries_plot.data)
fig.update_layout(height=700, legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()